In [1]:
import pandas as pd
import torch
#import dga_classifier.data as data
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
import string
import os
import tensorflow as tf

In [2]:
df3=pd.read_csv('../input/englishdataset/Dataset_English.csv', index_col=0)

In [3]:
df3

,Words,Label
0,aisha,name
1,aishah,name
2,ajay,name
3,aaisha,name
4,aaishah,name
...,...,...
568820,ayzlyn,name
568821,rozzlyn,name
568822,soffie,name
568823,valaysia,name


In [4]:
y=[]
words=[]
count_non_names = 0
count_names = 0
for x in range(len(df3)):
    if df3.iloc[x]['Label'] == 'non-name':
        y.append(0)
        count_non_names += 1
    else:
        y.append(1)
        count_names += 1
    words.append(str(df3.iloc[x]['Words']))

In [5]:
y = np.asarray(y) 

In [6]:
MAX_SEQUENCE_LENGTH = 31
EMBEDDING_DIM = 300
embeddings_index = {}
f = open('../input/glove6b/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [7]:
word_index = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}

In [8]:
embedding_matrix = np.zeros((27, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector 
    
print(embedding_matrix)
print(embedding_matrix.shape)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.29712     0.094049   -0.096662   ...  0.059717   -0.22853
   0.29602   ]
 [ 0.048902    0.29453    -0.06686    ... -1.46179998 -0.35120001
  -0.098858  ]
 ...
 [-0.32229    -0.28106001  0.51416999 ... -0.090831    0.43952
   0.51388001]
 [ 0.10812     0.11993    -0.27105999 ... -0.82524002 -0.17184
   0.51672   ]
 [-0.072908    0.42648     0.33423999 ... -0.43575001  0.57086003
   0.35189   ]]
(27, 300)


In [9]:
"""Train and test LSTM classifier"""
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
from keras.layers import Bidirectional


def build_model(max_features, maxlen):
    """Build Bi-LSTM model"""
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(max_features, 300, weights=[embedding_matrix], input_length=maxlen, trainable=True))
    model.add(tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(300)))
    model.add(tf.keras.layers.Dense(300))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    model.add(tf.keras.layers.Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

def run(max_epoch=25, nfolds=10, batch_size=128):
    """Run train/test on logistic regression model"""
    
    # Extract data and labels
    X = df3[['Words']]
    labels = df3[['Label']]
    valid_chars = dict(zip(string.ascii_lowercase, range(1,27)))
    print(len(valid_chars))
    max_features = len(valid_chars) + 1
    max_len=[]
    for x in words:
        max_len.append(len(x))
    maxlen = np.max(max_len)
    print(maxlen)

    X = [[valid_chars[y] for y in x] for x in words]
    print(len(X))
    X = sequence.pad_sequences(X, maxlen=maxlen)
    print(X.shape)
    
    weights={0: 0.41, 1: 0.59}
    
    final_data = []

    for fold in range(nfolds):
        print ("fold %u/%u" % (fold+1, nfolds))
        X_train, X_test, y_train, y_test, _, label_test = train_test_split(X, y, labels,test_size=0.2)
 
        print(X_train.shape)
        print ('Build model...')
        model = build_model(max_features, maxlen)

        print ("Train...")
        X_train, X_holdout, y_train, y_holdout = train_test_split(X_train, y_train, test_size=0.05)
        best_iter = -1
        best_auc = 0.0
        out_data = {}

        for ep in range(max_epoch):
            model.fit(X_train, y_train, batch_size=batch_size, epochs=1, class_weight=weights)

            t_probs = model.predict_proba(X_holdout)
            t_auc = sklearn.metrics.roc_auc_score(y_holdout, t_probs)

            print ('Epoch %d: auc = %f (best=%f)' % (ep, t_auc, best_auc))

            if t_auc > best_auc:
                best_auc = t_auc
                best_iter = ep

                probs = model.predict_proba(X_test)
                
                y_pred = model.predict(X_test)

                out_data = {'y':y_test, 'y_pred': y_pred, 'labels': label_test, 'probs':probs, 'epochs': ep,
                            'confusion_matrix': sklearn.metrics.confusion_matrix(y_test, probs > .5)}

                print (sklearn.metrics.confusion_matrix(y_test, probs > .5))
            else:
                # No longer improving...break and calc statistics
                if (ep-best_iter) > 2:
                    break

        final_data.append(out_data)
        model.save("model.h5")
        print("Saved model to disk")

        
    return final_data

In [10]:
"""Run experiments and create figs"""
import itertools
import os
import pickle
import matplotlib
matplotlib.use('Agg')
import numpy as np



from scipy import interp
from sklearn.metrics import roc_curve, auc

RESULT_FILE = 'results.pkl'

def run_experiments(islstm=True, nfolds=10):
    """Runs all experiments"""
    #bigram_results = None
    lstm_results = None

    #if isbigram:
    #    bigram_results = bigram.run(nfolds=nfolds)

    if islstm:
        lstm_results = run(nfolds=nfolds)

    return lstm_results

def create_figs(islstm=True, nfolds=10, force=False):
    """Create figures"""
    # Generate results if needed
    if force or (not os.path.isfile(RESULT_FILE)):
        lstm_results = run_experiments(islstm, nfolds)

        results = {'lstm': lstm_results}

        pickle.dump(results, open(RESULT_FILE, 'wb'))
    else:
        results = pickle.load(open(RESULT_FILE,'rb'))


    # xtract and calculate LSTM ROC
    if results['lstm']:
        lstm_results = results['lstm']
        fpr = []
        tpr = []
        for lstm_result in lstm_results:
            t_fpr, t_tpr, _ = roc_curve(lstm_result['y'], lstm_result['probs'])
            fpr.append(t_fpr)
            tpr.append(t_tpr)
        lstm_binary_fpr, lstm_binary_tpr, lstm_binary_auc = calc_macro_roc(fpr, tpr)

    # Save figure
    from matplotlib import pyplot as plt
    with plt.style.context('bmh'):
        plt.plot(lstm_binary_fpr, lstm_binary_tpr,
                 label='LSTM (AUC = %.4f)' % (lstm_binary_auc, ), rasterized=True)
        #plt.plot(bigram_binary_fpr, bigram_binary_tpr,
        #         label='Bigrams (AUC = %.4f)' % (bigram_binary_auc, ), rasterized=True)

        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=22)
        plt.ylabel('True Positive Rate', fontsize=22)
        plt.title('ROC - Binary Classification', fontsize=26)
        plt.legend(loc="lower right", fontsize=22)

        plt.tick_params(axis='both', labelsize=22)
        plt.savefig('results.png')

def calc_macro_roc(fpr, tpr):
    """Calcs macro ROC on log scale"""
    # Create log scale domain
    all_fpr = sorted(itertools.chain(*fpr))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(len(tpr)):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    return all_fpr, mean_tpr / len(tpr), auc(all_fpr, mean_tpr) / len(tpr)

if __name__ == "__main__":
    create_figs(nfolds=1) # Run with 1 to make it fast

26
31
568825
(568825, 31)
fold 1/1
(455060, 31)
Build model...
Train...
3378/3378 [==============================] - 63s 19ms/step - loss: 0.1465
Epoch 0: auc = 0.958518 (best=0.000000)
[[58902  7805]
 [ 4476 42582]]
3378/3378 [==============================] - 60s 18ms/step - loss: 0.1140
Epoch 1: auc = 0.965186 (best=0.958518)
[[57245  9462]
 [ 2406 44652]]
3378/3378 [==============================] - 60s 18ms/step - loss: 0.1004
Epoch 2: auc = 0.967843 (best=0.965186)
[[58854  7853]
 [ 3084 43974]]
3378/3378 [==============================] - 60s 18ms/step - loss: 0.0895
Epoch 3: auc = 0.969922 (best=0.967843)
[[58695  8012]
 [ 2704 44354]]
3378/3378 [==============================] - 59s 18ms/step - loss: 0.0796
Epoch 4: auc = 0.970483 (best=0.969922)
[[59926  6781]
 [ 3330 43728]]
3378/3378 [==============================] - 59s 18ms/step - loss: 0.0696
Epoch 5: auc = 0.970946 (best=0.970483)
[[60525  6182]
 [ 3690 43368]]
3378/3378 [==============================] - 59s 18ms/step

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:79: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead


In [11]:
file = open('results.pkl', 'rb')

# dump information to that file
data = pickle.load(file)

In [12]:
x=data['lstm'][0]['probs']
x1=data['lstm'][0]['y']

In [13]:
y_pred=[]
for i in x:
    if i > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [14]:
index1 = [i for i, x in enumerate(x1) if x == 1]
index0 = [i for i, x in enumerate(x1) if x == 0]

In [15]:
y_pred0=[]
y_pred1=[]
for i in index0:
    y_pred0.append(y_pred[i])
for i in index1:
    y_pred1.append(y_pred[i]) 

In [16]:
y_test0=[]
y_test1=[]
for i in index0:
    y_test0.append(x1[i])
for i in index1:
    y_test1.append(x1[i])

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test0, y_pred0))
print(accuracy_score(y_test1, y_pred1))
print(accuracy_score(x1, y_pred))

0.9073260677290239
0.9215861277572358
0.913224629719158


In [18]:
from sklearn.metrics import classification_report
print(classification_report(x1, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92     66707
           1       0.88      0.92      0.90     47058

    accuracy                           0.91    113765
   macro avg       0.91      0.91      0.91    113765
weighted avg       0.91      0.91      0.91    113765

